In [1]:
import dates_generation as dg
import os.path
import pandas as pd

Dates generation


In [2]:
def date_generation_process(month, year, holidays_list):
    holidays_list = dg.convert_string_to_date(holidays)
    days_list = dg.weekdays_list(month, year)
    business_days_list = dg.generate_date_from_list(month, year, days_list)
    real_business = dg.extract_holidays_from_weekdays(business_days_list, holidays_list)
    list_of_dates = dg.create_list_of_dates(real_business)
    dictionary = dg.create_dataframe_dictionary(list_of_dates)
    return dictionary

In [3]:
holidays = ["05-05-23", "04-05-23"]

In [4]:
dates_dictionary = date_generation_process(5, 2023, holidays)

2023-05-01 00:00:00
2023-05-02 00:00:00
2023-05-03 00:00:00
2023-05-06 00:00:00
2023-05-08 00:00:00
2023-05-09 00:00:00
2023-05-10 00:00:00
2023-05-11 00:00:00
2023-05-12 00:00:00
2023-05-13 00:00:00
2023-05-15 00:00:00
2023-05-16 00:00:00
2023-05-17 00:00:00
2023-05-18 00:00:00
2023-05-19 00:00:00
2023-05-20 00:00:00
2023-05-22 00:00:00
2023-05-23 00:00:00
2023-05-24 00:00:00
2023-05-25 00:00:00
2023-05-26 00:00:00
2023-05-27 00:00:00
2023-05-29 00:00:00
2023-05-30 00:00:00
2023-05-31 00:00:00


In [5]:
dates_dictionary["01-05-2023"]

,articulo,descripcion,chess,wms,fisico


Dataframe base ABC clasification


In [6]:
import dataframe_class_base as dcb

In [7]:
# read ABC base file
fullpath = os.path.abspath(".//abc_mayo_23.xlsx")

df_clasificacion = pd.read_excel(fullpath, sheet_name="Hoja1")

In [8]:
df_abc = dcb.cantidad_segun_abc(df_clasificacion)

In [9]:
df_abc

,articulo,descripcion,clasificacion,cantidad
0,7038,BRAHMA - X12 1000RET#,A,3
1,25668,PEPSI NF PET X6 500CC,A,3
2,7634,BRAHMA - 4X6 LAT 473,A,3
3,25644,7-UP NF PET X6 500CC,A,3
4,7026,QUILMES - X12 1000RET,A,3
...,...,...,...,...
318,78068,STA MAZO DE CARTAS BUDWEISER,C,1
320,78131,"VINO MALBEC MISTERIO X6 0,750 L",C,1
321,78141,TOSTADAS DE ARROZ MOLINOS ALA X24 150GR,C,1
325,78233,YERBA MATE CACHAMATE X12 500GR,C,1


DB Operations


In [10]:
import sqlite_db_generation as sqlgen
import sqlite3
import math

In [11]:
sqlgen.create_db("abc_quantity.db", "abc")

In [12]:
sqlgen.insert_article_df_into_db(df_abc, "abc_quantity.db", "abc")

In [13]:
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")

In [14]:
sqlgen.remove_quant_for_article("abc_quantity.db", "abc",25668)

In [15]:
sqlgen.select_an_article("abc_quantity.db", "abc", 25668)

(25668, 'PEPSI NF PET X6 500CC', 'A', 2)


In [16]:
random_art = sqlgen.select_random_article("abc_quantity.db", "abc", "A")

In [17]:
sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", random_art[0])

True

In [18]:
def get_max_per_category_per_day(db_name, table_name, dates_dictionary, category):
    cnt = sqlite3.connect(db_name)
    business_days = len(dates_dictionary.keys())
    query = cnt.execute(
        f""" SELECT COUNT(*)
            FROM {table_name}
            WHERE CLASIFICACION = '{category}'; """
    )

    if(category == "A"):
        quantity = query.fetchone()[0] * 3
    elif(category == "B"):
        quantity = query.fetchone()[0] * 2
    else:
        quantity = query.fetchone()[0] * 1

    return math.ceil(quantity/business_days)
    

In [19]:
# def add_article_to_dataframe(cod,descrip,dictionary,dictionary_key):
#     if cod in dictionary[dictionary_key].values:
#         return
#     else:
#         df_aux = {'articulo': cod, 'descripcion': descrip, 'chess': "", 'wms': "", 'fisico': "" }
#         dictionary[dictionary_key] = dictionary[dictionary_key].append(df_aux, ignore_index = True)
#     return dictionary

In [20]:
def add_article_to_dataframe(cod,descrip,date_df):
    df_aux = {'articulo': cod, 'descripcion': descrip, 'chess': "", 'wms': "", 'fisico': "" }
    date_df = pd.concat([date_df, pd.DataFrame([df_aux])], ignore_index=True)
    return date_df

In [21]:
def check_article_already_in_df(cod, date_df):
    if cod in date_df.values:
        print(date_df.values)
        return True
    else:
        return False

In [22]:
a_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"A")
b_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"B")
c_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"C")


In [23]:
def fill_date(date_df, maxA,maxB,maxC,selected_category):
    if(selected_category == "A"):
        contador = 0
        while(contador < maxA):
            if(sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", selected_category)):
                rnd_art = sqlgen.select_random_article("abc_quantity.db", "abc", selected_category)
                if(check_article_already_in_df(rnd_art[0],date_df)):
                    #print(rnd_art[0], "already in df")
                    continue
                else:
                    if(sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", rnd_art[0])):
                        date_df = add_article_to_dataframe(rnd_art[0],rnd_art[1],date_df)
                        sqlgen.remove_quant_for_article("abc_quantity.db", "abc",rnd_art[0])
                        contador+=1
                    else:
                        continue
            else:
                break
        return date_df
    elif(selected_category == "B"):
        contador = 0
        while(contador < maxB):
            if(sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", selected_category)):
                rnd_art = sqlgen.select_random_article("abc_quantity.db", "abc", selected_category)
                if(check_article_already_in_df(rnd_art[0],date_df)):
                    #print(rnd_art[0], "already in df")
                    continue
                else:
                    if(sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", rnd_art[0])):
                        date_df = add_article_to_dataframe(rnd_art[0],rnd_art[1],date_df)
                        sqlgen.remove_quant_for_article("abc_quantity.db", "abc",rnd_art[0])
                        contador+=1
                    else:
                        continue
            else:
                break
        return date_df
    else:
        contador = 0
        while(contador < maxC):
            if(sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", selected_category)):
                rnd_art = sqlgen.select_random_article("abc_quantity.db", "abc", selected_category)
                if(check_article_already_in_df(rnd_art[0],date_df)):
                    #print(rnd_art[0], "already in df")
                    continue
                else:
                    if(sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", rnd_art[0])):
                        date_df = add_article_to_dataframe(rnd_art[0],rnd_art[1],date_df)
                        sqlgen.remove_quant_for_article("abc_quantity.db", "abc",rnd_art[0])
                        contador+=1
                    else:
                        continue
            else:
                break
        return date_df        

In [24]:
#reset dates dictionary
dates_dictionary = date_generation_process(5, 2023, holidays)

2023-05-01 00:00:00
2023-05-02 00:00:00
2023-05-03 00:00:00
2023-05-06 00:00:00
2023-05-08 00:00:00
2023-05-09 00:00:00
2023-05-10 00:00:00
2023-05-11 00:00:00
2023-05-12 00:00:00
2023-05-13 00:00:00
2023-05-15 00:00:00
2023-05-16 00:00:00
2023-05-17 00:00:00
2023-05-18 00:00:00
2023-05-19 00:00:00
2023-05-20 00:00:00
2023-05-22 00:00:00
2023-05-23 00:00:00
2023-05-24 00:00:00
2023-05-25 00:00:00
2023-05-26 00:00:00
2023-05-27 00:00:00
2023-05-29 00:00:00
2023-05-30 00:00:00
2023-05-31 00:00:00


In [25]:
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")

In [26]:
sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", "A")

True

In [27]:
for i in dates_dictionary:
    print("with As")
    dates_dictionary[i] = fill_date(dates_dictionary[i],a_articles_per_day, b_articles_per_day,c_articles_per_day,"A")
for i in dates_dictionary:
    print("with Bs")
    dates_dictionary[i] = fill_date(dates_dictionary[i],a_articles_per_day, b_articles_per_day,c_articles_per_day,"B")
for i in dates_dictionary:
    print("with Cs")
    dates_dictionary[i] = fill_date(dates_dictionary[i],a_articles_per_day, b_articles_per_day,c_articles_per_day,"C")

with As
with As
with As
with As
with As
with As
with As
with As
with As
[[1416 'GATORADE MANZANA * X6 PET 500' '' '' '']
 [25352 'BRAHMA DORADA CAN 4X6 473CC' '' '' '']
 [7634 'BRAHMA - 4X6 LAT 473' '' '' '']]
with As
with As
[[7634 'BRAHMA - 4X6 LAT 473' '' '' '']
 [25352 'BRAHMA DORADA CAN 4X6 473CC' '' '' '']
 [25644 '7-UP NF PET X6 500CC' '' '' '']]
with As
with As
with As
with As
with As
with As
[[24880 'STELLA ARTOIS PM RET X12 975CC' '' '' '']
 [24012 'BRAHMA DORADA OW X6 710CC' '' '' '']]
with As
with As
[[19337 '7-UP PET X6 1500 NF' '' '' '']
 [25194 'MIRINDA NARANJA PET X6 500CC' '' '' '']
 [7478 'MIRINDA NARANJA * X6 PET1500#' '' '' '']]
with As
[[25194 'MIRINDA NARANJA PET X6 500CC' '' '' '']
 [7038 'BRAHMA - X12 1000RET#' '' '' '']
 [19086 'PEPSI PET X6 1500 NF' '' '' '']]
with As
[[11522 'GATORADE FRU TRO * X6 PET 500#' '' '' '']
 [7038 'BRAHMA - X12 1000RET#' '' '' '']
 [19086 'PEPSI PET X6 1500 NF' '' '' '']]
[[11522 'GATORADE FRU TRO * X6 PET 500#' '' '' '']
 [7038 'BR

KeyboardInterrupt: 

In [ ]:
for i in dates_dictionary:
    print(i)

In [ ]:
dates_dictionary.values()

In [ ]:
dates_dictionary['22-05-2023']